In [ ]:
!pip install requests beautifulsoup4
!pip install apimoex
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 7.6 MB/s eta 0:00:00


Со странице https://fin-plan.org/lk/actions/all/ получаем все ссылки ведущие к описанию эмитента.

In [ ]:
import apimoex
import requests
import re
import multiprocessing
import copy
import os
import traceback
import boto3
import io
import json

import pandas as pd
import numpy as np

from tqdm import tqdm
from bs4 import BeautifulSoup
from botocore.exceptions import ClientError, NoCredentialsError
from traceback import format_exc

In [ ]:
url = "https://fin-plan.org/lk/actions/all/"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}
response = requests.get(url, headers=headers)
if response.status_code == 200:
    # получаем html страницы
    soup = BeautifulSoup(response.text, 'html.parser')
    # находим ссылки
    links = soup.find_all('a', href=True)
    # прописываем паттерн для поиска ссылок
    pattern = re.compile(r'/lk/actions/[0-9A-Z]+/')
    found_links = []
    for link in links:
        href = link['href']
        # проверяем паттерн
        if pattern.match(href):
            found_links.append(href)
else:
    print(f"Ошибка при получении страницы: {response.status_code}")

print(f'Найдено secid: {len(found_links)} шт.')

366


После чего по каждой из этих ссылок делаем запрос и получаем КОД бумаги, то есть её уникальное обозначения для дальнейшего использования.

In [ ]:
found_secids = []

def get_secids(num, actions, mem_secids):
    for action in tqdm(actions, desc=f"Получение каждого потока {num + 1}", unit="secid"):
        url = f'https://fin-plan.org{action}'
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            # получаем html страницы
            soup = BeautifulSoup(response.text, 'html.parser')
            # находим h1 заголовок
            h1_names = soup.find_all('h1')
            # паттерн для тикета secid
            pattern = re.compile(r'\(.+\)')
            for name in h1_names:
                # находим все совпадения
                finded = re.findall(pattern, str(name))
                for find in finded:
                    # добавляем найденный тикет
                    mem_secids.append(find[1:-1])
        else:
            print(f"Ошибка при получении страницы: {response.status_code}")


processes = []
threads = 10
with multiprocessing.Manager() as manager:
    mem_secids = manager.list()
    for i in range(threads):
        p = multiprocessing.Process(target=get_secids, args=(i, found_links[i::threads], mem_secids,))
        processes.append(p)
        p.start()

    for p in processes:
        p.join()
    found_secids = list(mem_secids)

print(found_secids)

Получение каждого потока 6: 100%|██████████| 37/37 [12:31<00:00, 20.32s/secid]


['AVAZP', 'ALNU', 'ACKO', 'AVAZ', 'ABRD', 'AVAN', 'APRI', 'ALRS', 'KLVZ', 'AKRN', 'DATA', 'AMEZ', 'ASSB', 'RU000A100EP4', 'APTK', 'ARSA', 'AFLT', 'RU000A1007R9', 'ASTR', 'ARMD', 'BSPBP', 'BANE', 'ALBK', 'ABBN', 'BSPB', 'BANEP', 'BLNG', 'FTRE', 'BISV', 'BISVP', 'VSEH', 'RU000A0JR2A5', 'VLHZ', 'VSMO', 'BRZL', 'WUSH', 'VTBR', 'RU000A0JR2C1', 'RU000A0JR282', 'VTRS', 'VJGZ', 'VGSBP', 'GAZA', 'VDSB', 'VJGZP', 'VSYDP', 'VZRZ', 'VGSB', 'VZRZP', 'VSYD', 'GAZT', 'GRNT', 'RTGZ', 'GAZAP', 'GAZS', 'HALS', 'DIAS', 'DASB', 'GAZC', 'GTSS', 'GTRK', 'GAZP', 'GMKN', 'SIBN', 'GTPR', 'FESH', 'DVEC', 'DIXY', 'DERZP', 'DSKY', 'EUTR', 'IVAT', 'ZMZN', 'UTSY', 'RUSI', 'EELT', 'LEAS', 'RU000A0JUQZ6', 'ZVEZ', 'ZILL', 'ZMZNP', 'RU000A0JTXM2', 'RU000A0JUHS0', 'ZAYM', 'AQUA', 'INGR', 'IRKT', 'KTSBP', 'RU000A0JUW49', 'RU000A0JVEZ0', 'RU000A0JVT35', 'RU000A0JUGA0', 'KMAZ', 'KTSB', 'IGSTP', 'IGST', 'IDVP', 'KMTZ', 'KZMS', 'KLSB', 'TGKD', 'KUNF', 'KOGK', 'KCHE', 'IRAO', 'IRGZ', 'KMEZ', 'KGKC', 'TGKDP', 'DELI', 'KSGR', '

Затем уже получаем по каждому secid данные с мосбиржи.

In [ ]:
data_frames = {}
remove_secids = []
for secid in tqdm(found_secids, desc="Получение каждого", unit="secid"):
    with requests.Session() as session:
        # получение данных по secid
        data = apimoex.get_board_history(session, secid, columns=['TRADEDATE', 'SHORTNAME', 'SECID', 'OPEN', 'LOW', 'HIGH', 'CLOSE', 'VOLUME', 'VALUE'])
        df = pd.DataFrame(data)
        # проверка, если не нашел.
        if df.empty:
            # дополняем список для удаления
            remove_secids.append(secid)
            continue
        # получаем пропуски
        miss = df[df.isnull().any(axis=1) == True]
        # удаляем строки с пропусками
        df_cleaned = df.dropna()
        # обновляем индексы после удаления
        df_clean = df_cleaned.reset_index(drop=True)
        # дату переделываем в соответствующий тип данных
        df_clean['TRADEDATE'] = pd.to_datetime(df_clean['TRADEDATE'])
        # проверка, если все строки были пустыми
        if df_clean.empty:
            # дополняем список для удаления
            remove_secids.append(secid)
            continue
        # получаем первую строку
        row = df_clean.iloc[0]
        # преобразуем в хэш, где ключом будет secid, а в его значении хэш с именем, данными, процентом и индексом пропусков и числом строк и пропусков
        data_frames[row['SECID']] = {
            'short_name': row['SHORTNAME'],
            'data_frame': df_clean.drop(['SHORTNAME', 'SECID'], axis=1),
            'miss_percent': miss.shape[0] / df_clean.shape[0] * 100,
            'miss_count': miss.shape[0],
            'row_count': df_clean.shape[0],
            'miss_index': miss.index
        }

In [ ]:
data_frames = {}
remove_secids = []

def get_moex_data(num, secids, mem_data, mem_remove):
    for secid in tqdm(secids, desc=f"Получение каждого потока {num + 1}", unit="secid data"):
        with requests.Session() as session:
            # получение данных по secid
            data = apimoex.get_board_history(session, secid, columns=['TRADEDATE', 'SHORTNAME', 'SECID', 'OPEN', 'LOW', 'HIGH', 'CLOSE', 'VOLUME', 'VALUE'])
            df = pd.DataFrame(data)
            # проверка, если не нашел.
            if df.empty:
                # дополняем список для удаления
                mem_remove.append(secid)
                continue
            # получаем пропуски
            miss = df[df.isnull().any(axis=1) == True]
            # проверка, если все строки были пустыми
            if df.dropna().empty:
                # дополняем список для удаления
                mem_remove.append(secid)
                continue
            # дату переделываем в соответствующий тип данных
            df['TRADEDATE'] = pd.to_datetime(df['TRADEDATE'])
            # получаем первую строку
            row = df.iloc[0]
            # преобразуем в хэш, где ключом будет secid, а в его значении хэш с именем, данными, процентом и индексом пропусков и числом строк и пропусков
            mem_data[row['SECID']] = {
                'short_name': row['SHORTNAME'],
                'data_frame': df.drop(['SHORTNAME', 'SECID'], axis=1),
                'miss_percent': miss.shape[0] / df.shape[0] * 100,
                'miss_count': miss.shape[0],
                'row_count': df.shape[0],
                'miss_index': miss.index
            }


processes = []
threads = 10
with multiprocessing.Manager() as manager:
    mem_data = manager.dict()
    mem_remove = manager.list()
    for i in range(threads):
        p = multiprocessing.Process(target=get_moex_data, args=(i, found_secids[i::threads], mem_data, mem_remove,))
        processes.append(p)
        p.start()

    for p in processes:
        p.join()
    data_frames = dict(mem_data)
    remove_secids = list(mem_remove)

print(len(remove_secids), len(data_frames))

Получение каждого потока 4: 100%|██████████| 37/37 [02:49<00:00,  4.59s/secid data]


34 332


Не все удалось найти, поэтому можем их удалить и сравнить с нашими сохраненными.

In [ ]:
found_secids = [item for item in found_secids if item not in remove_secids]
print([secid for secid in found_secids if secid not in data_frames.keys()])

[]


Для работы S3 необходимо установить ключи доступа. Будьте аккуратны, их никто знать не должен.

In [ ]:
os.environ['AWS_ACCESS_KEY_ID'] = <access_key>
os.environ['AWS_SECRET_ACCESS_KEY'] = <secret_access_key>

Теперь можем сохранить данные.

In [104]:
BUCKET = 'russian-stocks-quotes'

access_key = os.getenv('AWS_ACCESS_KEY_ID')
secret_key = os.getenv('AWS_SECRET_ACCESS_KEY')
endpoint_url = 'https://storage.yandexcloud.net'

# Создание клиента S3
s3_client = boto3.client('s3',
                         region_name='ru-central1',
                         aws_access_key_id=access_key,
                         aws_secret_access_key=secret_key,
                         endpoint_url=endpoint_url)

def upload_object_to_s3(key, body):
    response = s3_client.put_object(Bucket=BUCKET, Key=key, Body=body)
    if response['ResponseMetadata']['HTTPStatusCode'] == 200:
        print(f"Успешно сохранен в {BUCKET}/{key}")
    else:
        print(f"Ошибка при сохранении: {response['ResponseMetadata']['HTTPStatusCode']}")

def upload_data_frame_to_s3(secid, data_frame, dir):
    pickle_buffer = io.BytesIO()
    data_frame.to_pickle(pickle_buffer)
    pickle_buffer.seek(0)
    data_frame_file = f'{dir}secids/{secid}/{secid}_data_frame.pkl'
    upload_object_to_s3(data_frame_file, pickle_buffer)

def upload_info_to_s3(secid, info, dir):
    info['miss_index'] = info['miss_index'].to_numpy().tolist()
    json_data = json.dumps(info)
    info_file = f'{dir}secids/{secid}/{secid}_info.pkl'
    upload_object_to_s3(info_file, json_data)

def upload_secid_names(dict_data, dir):
    json_data = json.dumps(list(dict_data.keys()))
    secid_names_file = f'{dir}secid_names.pkl'
    upload_object_to_s3(secid_names_file, json_data)

def upload_data_to_s3(dict_data, dir):
    try:
        upload_secid_names(dict_data, dir)
        for secid, data in dict_data.items():
            copy_data = copy.deepcopy(data)
            upload_data_frame_to_s3(secid, copy_data['data_frame'], dir)
            del copy_data['data_frame']
            upload_info_to_s3(secid, copy_data, dir)
    except ClientError as e:
        print(f"Произошла ошибка: {e.response['Error']['Message']}")
    except Exception as e:
        error_message = f"Неизвестная ошибка: {str(e)}"
        error_context = traceback.format_exc()
        print(f"{error_message}\nКонтекст ошибки:\n{error_context}")

def list_directories(s3_client):
    directories = set()
    try:
        paginator = s3_client.get_paginator('list_objects_v2')
        for page in paginator.paginate(Bucket=BUCKET, Delimiter='/'):
            for prefix in page.get('CommonPrefixes', []):
                directories.add(prefix.get('Prefix'))
    except NoCredentialsError:
        print("Ошибка: Неверные учетные данные.")
    except Exception as e:
        print(f"Произошла ошибка: {e}")
    return directories

def download_object_from_s3(key):
    response = s3_client.get_object(Bucket=BUCKET, Key=key)
    if response['ResponseMetadata']['HTTPStatusCode'] == 200:
        print(f"Успешно получен из {BUCKET}/{key}")
    else:
        print(f"Ошибка при получении: {response['ResponseMetadata']['HTTPStatusCode']}")
    return response['Body'].read()

def download_info_from_s3(dir, secid):
    key = f'{dir}secids/{secid}/{secid}_info.pkl'
    response = download_object_from_s3(key)
    data = json.loads(response)
    data['miss_index'] = pd.Index(np.array(data['miss_index']))
    return data

def download_data_frame_from_s3(dir, secid):
    key = f'{dir}secids/{secid}/{secid}_data_frame.pkl'
    response = download_object_from_s3(key)
    buffer = io.BytesIO(response)
    data = pd.read_pickle(buffer)
    data['TRADEDATE'] = pd.to_datetime(data['TRADEDATE'])
    return data

def download_secid_names(dir):
    key = f'{dir}secid_names.pkl'
    return json.loads(download_object_from_s3(key))

def download_data_from_s3(dir, secids=None):
    data = {}
    try:
        directories = download_secid_names(dir)
        for secid in directories:
            if secids is None or secid in secids:
                if secids is not None:
                    secids.remove(secid)
                data[secid] = download_info_from_s3(dir, secid)
                data[secid]['data_frame'] = download_data_frame_from_s3(dir, secid)
    except Exception as e:
        error_message = f"Неизвестная ошибка: {str(e)}"
        error_context = traceback.format_exc()
        print(f"{error_message}\nКонтекст ошибки:\n{error_context}")
    if secids is not None and len(secids) > 0:
        print(f'Не нашли {secids}')
    return data

In [67]:
upload_data_to_s3(data_frames, 'data_without_preprocessing/')

Успешно сохранен в russian-stocks-quotes/data_without_preprocessing/secid_names.pkl
Успешно сохранен в russian-stocks-quotes/data_without_preprocessing/secids/APRI/APRI_data_frame.pkl
Успешно сохранен в russian-stocks-quotes/data_without_preprocessing/secids/APRI/APRI_info.pkl
Успешно сохранен в russian-stocks-quotes/data_without_preprocessing/secids/KLVZ/KLVZ_data_frame.pkl
Успешно сохранен в russian-stocks-quotes/data_without_preprocessing/secids/KLVZ/KLVZ_info.pkl
Успешно сохранен в russian-stocks-quotes/data_without_preprocessing/secids/ASTR/ASTR_data_frame.pkl
Успешно сохранен в russian-stocks-quotes/data_without_preprocessing/secids/ASTR/ASTR_info.pkl
Успешно сохранен в russian-stocks-quotes/data_without_preprocessing/secids/ACKO/ACKO_data_frame.pkl
Успешно сохранен в russian-stocks-quotes/data_without_preprocessing/secids/ACKO/ACKO_info.pkl
Успешно сохранен в russian-stocks-quotes/data_without_preprocessing/secids/AVAZP/AVAZP_data_frame.pkl
Успешно сохранен в russian-stocks-quot

И получение данных.

In [99]:
downloaded_data = download_data_from_s3('data_without_preprocessing/', secids=['YDEX', 'ALBK', 'QQQQ'])

Успешно получен из russian-stocks-quotes/data_without_preprocessing/secid_names.pkl
Успешно получен из russian-stocks-quotes/data_without_preprocessing/secids/ALBK/ALBK_info.pkl
Успешно получен из russian-stocks-quotes/data_without_preprocessing/secids/ALBK/ALBK_data_frame.pkl
Успешно получен из russian-stocks-quotes/data_without_preprocessing/secids/YDEX/YDEX_info.pkl
Успешно получен из russian-stocks-quotes/data_without_preprocessing/secids/YDEX/YDEX_data_frame.pkl
Не нашли ['QQQQ']


Пример получаемых данных.

In [101]:
downloaded_data['YDEX']

{'short_name': 'ЯНДЕКС',
 'miss_percent': 13.953488372093023,
 'miss_count': 12,
 'row_count': 86,
 'miss_index': Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], dtype='int64'),
 'data_frame':     TRADEDATE    OPEN     LOW    HIGH   CLOSE   VOLUME         VALUE
 0  2024-07-08     NaN     NaN     NaN     NaN        0  0.000000e+00
 1  2024-07-09     NaN     NaN     NaN     NaN        0  0.000000e+00
 2  2024-07-10     NaN     NaN     NaN     NaN        0  0.000000e+00
 3  2024-07-11     NaN     NaN     NaN     NaN        0  0.000000e+00
 4  2024-07-12     NaN     NaN     NaN     NaN        0  0.000000e+00
 ..        ...     ...     ...     ...     ...      ...           ...
 81 2024-10-29  3765.0  3670.0  3792.0  3758.0  1097054  4.104354e+09
 82 2024-10-30  3769.0  3685.5  3817.5  3710.0   730830  2.751687e+09
 83 2024-10-31  3685.0  3621.0  3734.5  3666.5   763630  2.815400e+09
 84 2024-11-01  3675.0  3622.0  3700.0  3674.0   797602  2.917418e+09
 85 2024-11-02  3681.5  3652.5  3694.0  

In [96]:
downloaded_data['ALBK']

{'short_name': 'БестЭфБ ао',
 'miss_percent': 52.10583153347732,
 'miss_count': 965,
 'row_count': 1852,
 'miss_index': Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
        ...
        1255, 1262, 1264, 1265, 1273, 1311, 1741, 1849, 1850, 1851],
       dtype='int64', length=965),
 'data_frame':       TRADEDATE   OPEN    LOW   HIGH  CLOSE  VOLUME   VALUE
 0    2014-06-09    NaN    NaN    NaN    NaN       0       0
 1    2014-06-10    NaN    NaN    NaN    NaN       0       0
 2    2014-06-11    NaN    NaN    NaN    NaN       0       0
 3    2014-06-16    NaN    NaN    NaN    NaN       0       0
 4    2014-06-17    NaN    NaN    NaN    NaN       0       0
 ...         ...    ...    ...    ...    ...     ...     ...
 1847 2021-10-05  110.0  102.5  110.0  108.0    3760  398425
 1848 2021-10-06  103.5  103.5  108.5  106.5     830   88025
 1849 2021-10-07    NaN    NaN    NaN    NaN       0       0
 1850 2021-10-08    NaN    NaN    NaN    NaN       0       0
 1851 2021-1

Получение всех данных.

In [105]:
full_data = download_data_from_s3('data_without_preprocessing/')

Успешно получен из russian-stocks-quotes/data_without_preprocessing/secid_names.pkl
Успешно получен из russian-stocks-quotes/data_without_preprocessing/secids/APRI/APRI_info.pkl
Успешно получен из russian-stocks-quotes/data_without_preprocessing/secids/APRI/APRI_data_frame.pkl
Успешно получен из russian-stocks-quotes/data_without_preprocessing/secids/KLVZ/KLVZ_info.pkl
Успешно получен из russian-stocks-quotes/data_without_preprocessing/secids/KLVZ/KLVZ_data_frame.pkl
Успешно получен из russian-stocks-quotes/data_without_preprocessing/secids/ASTR/ASTR_info.pkl
Успешно получен из russian-stocks-quotes/data_without_preprocessing/secids/ASTR/ASTR_data_frame.pkl
Успешно получен из russian-stocks-quotes/data_without_preprocessing/secids/ACKO/ACKO_info.pkl
Успешно получен из russian-stocks-quotes/data_without_preprocessing/secids/ACKO/ACKO_data_frame.pkl
Успешно получен из russian-stocks-quotes/data_without_preprocessing/secids/AVAZP/AVAZP_info.pkl
Успешно получен из russian-stocks-quotes/dat

In [106]:
full_data['SBER']['data_frame'].head()

,TRADEDATE,OPEN,LOW,HIGH,CLOSE,VOLUME,VALUE
0,2013-03-25,96.00,96.00,101.14,98.79,593680,59340002.8
1,2013-03-26,98.58,97.08,99.31,97.20,1283550,126030358.8
2,2013-03-27,97.90,95.39,98.00,96.75,1261950,121835900.2
3,2013-03-28,96.38,95.72,98.66,98.59,1971410,192469794.9
4,2013-03-29,98.60,98.32,99.09,98.76,782000,77268860.1
